In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import PIL
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow import keras
tf.experimental.numpy.experimental_enable_numpy_behavior()

In [3]:
# Loading the model
model_path = "/content/drive/MyDrive/PAML Project: Handwriting to Text/prediction_model.keras"
model = tf.keras.models.load_model(model_path)

In [4]:
def distortion_free_resize(image, img_size):
  w, h = img_size
  image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)

  # Check tha amount of padding needed to be done.
  pad_height = h - tf.shape(image)[0]
  pad_width = w - tf.shape(image)[1]

  # only necessary if you want to do same amount of padding on both sides.
  if pad_height % 2 != 0:
    height = pad_height // 2
    pad_height_top = height +1
    pad_height_bottom = height
  else:
    pad_height_top = pad_height_bottom = pad_height // 2

  if pad_width % 2 != 0:
    width = pad_width // 2
    pad_width_left = width + 1
    pad_width_right = width
  else:
    pad_width_left = pad_width_right = pad_width // 2

  image = tf.pad(
      image, paddings=[
          [pad_height_top, pad_height_bottom],
          [pad_width_left, pad_width_right],
          [0, 0],
      ],
  )
  image = tf.transpose(image, perm=[1,0,2])
  image = tf.image.flip_left_right(image)
  return image

image_width = 128
image_height = 32

def preprocess_image(img, img_size=(image_width, image_height)):
  image = Image.open(image_path)
  image = tf.io.read_file(image_path)
  image = tf.image.decode_png(image, 1)
  image = distortion_free_resize(image, img_size)
  image = tf.cast(image, tf.float32) / 255.0
  image = np.expand_dims(image, axis=0)
  return image

In [5]:
import pickle
with open("/content/drive/MyDrive/PAML Project: Handwriting to Text/characters", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
    print(b)
AUTOTUNE = tf.data.AUTOTUNE

# Maping characaters to integers
char_to_num = StringLookup(vocabulary=b, mask_token=None)

#Maping integers back to original characters
num_to_chars = StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)

['L', 'E', '"', '0', '2', 's', 'l', 'w', 'p', 'P', 'H', '/', 'O', 'V', 'f', '!', 'X', 'v', 'M', ':', 'N', '4', '3', "'", 'F', 'J', '(', 'K', '+', 'o', '#', 'I', 'r', '9', ',', '&', '8', 'B', '*', 'Q', 'g', '.', 'W', 'j', 'q', 'm', 'c', '5', ';', '7', 'U', 'h', 'u', 'a', ')', 'R', 'i', 'C', 'z', 'n', '-', 'Y', 'D', 'x', 'd', 'S', 'y', 'Z', 'e', 'k', 'G', '?', 'T', 'A', '6', 't', '1', 'b']


In [6]:
# A utility function to decode the output of the network
def decode_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search.
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0]

    # Iterate over the results and get back the text.
    output_text = []

    for res in results:
        res = tf.gather(res, tf.where(tf.math.not_equal(res, -1)))
        res = tf.strings.reduce_join(num_to_chars(res)).numpy().decode("utf-8")
        output_text.append(res)

    return output_text

In [7]:
# Function to predict handwritten text
def predict(image_path, model):
    # Choose device, GPU if available
    with tf.device('/gpu:0'):  # This will try to use the first GPU
        preprocessed_image = preprocess_image(image_path, (128, 32))  # Adjust dimensions as needed for your model
        predictions = model.predict(preprocessed_image)
        predicted_text = decode_predictions(predictions)
    return predicted_text


In [11]:
path1 = "/content/WhatsApp Image 2024-04-17 at 9.04.50 PM.jpg"
path2 = "/content/WhatsApp Image 2024-04-17 at 9.05.08 PM.png"
path3 = "/content/WhatsApp Image 2024-04-17 at 9.05.26 PM.png"
path4 = "/content/WhatsApp Image 2024-04-17 at 9.15.20 PM.png"
path5 = "/content/WhatsApp Image 2024-04-17 at 9.16.42 PM.png"
path6 = "/content/WhatsApp Image 2024-04-17 at 9.18.03 PM.png"
path7 = "/content/WhatsApp Image 2024-04-17 at 9.20.34 PM.png"
path8 = "/content/WhatsApp Image 2024-04-17 at 9.24.06 PM.png"
path9 = "/content/WhatsApp Image 2024-04-17 at 9.43.06 PM.png"
path10 = "/content/WhatsApp Image 2024-04-17 at 9.45.46 PM.png"
path11 = "/content/WhatsApp Image 2024-04-17 at 9.47.30 PM.png"
path12 = "/content/WhatsApp Image 2024-04-17 at 9.50.16 PM.png"
path13 = "/content/WhatsApp Image 2024-04-17 at 9.52.49 PM.jpg"
path14 = "/content/WhatsApp Image 2024-04-17 at 9.54.07 PM.jpg"

In [9]:
!pip install Pillow

In [13]:
# Predict text from an image
image_path = path13  # Replace with your handwritten image path
predicted_text = predict(image_path, model)

# Display results
plt.imshow(Image.open(image_path))
plt.title('Handwritten Input')
plt.axis('off')
plt.show()


print('Predicted Typed Text:', predicted_text)

FileNotFoundError: [Errno 2] No such file or directory: '/content/WhatsApp Image 2024-04-17 at 9.52.49 PM.png'